<a href="https://colab.research.google.com/github/DiaaZiada/Nazra/blob/master/NazRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from time import time
import torch
from torch import nn

In [0]:
class Conv(nn.Module):
    
    def __init__(self, in_f, k, s, out_f, p):
        super(Conv, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_f, out_channels=out_f, kernel_size=k, stride=s, padding=p)
        self.bn = nn.BatchNorm2d(out_f)
    
    def forward(self, x):
        
        x = self.conv(x)
        x = self.bn(x)
        
        return x

In [0]:
class ResId(nn.Module):
    
    def __init__(self, prev, f1, f2):
        super(ResId, self).__init__()
        
        self.conv1 = Conv(in_f=prev, out_f=f1, k=1, s=1, p=0)
        self.conv2 = Conv(in_f=f1, out_f=f1, k=3, s=1, p=1)
        self.conv3 = Conv(in_f=f1, out_f=f2, k=1, s=1, p=0)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        res = x
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        
        x += res
        
        x = self.relu(x)
        
        return x
        

In [0]:
class ResConv(nn.Module):
    
    def __init__(self, prev, s, f1, f2):
        super(ResConv, self).__init__()
        
        self.conv1 = Conv(in_f=prev, out_f=f1, k=1, s=s, p=0)
        self.conv2 = Conv(in_f=f1, out_f=f1, k=3, s=1, p=1)
        self.conv3 = Conv(in_f=f1, out_f=f2, k=1, s=1, p=0)
        
        self.res = Conv(in_f=prev, out_f=f2, k=1, s=s, p=0)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        res = self.res(x)
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        
        x += res
        
        x = self.relu(x)
        
        return x
        

In [0]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = Conv(in_f=4, out_f=64, k=7, s=1, p=1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.res2a = ResConv(prev=64, s=1, f1=64, f2=256)
        self.res2b = ResId(prev=256, f1=64, f2=256)
        self.res2c = ResId(prev=256, f1=64, f2=256)
        
        self.res3a = ResConv(prev=256, s=2, f1=128, f2=512)
        self.res3b = ResId(prev=512, f1=128, f2=512)
        self.res3c = ResId(prev=512, f1=128, f2=512)
        
        self.res4a = ResConv(prev=512, s=2, f1=256, f2=1024)
        self.res4b = ResId(prev=1024, f1=256, f2=1024)
        self.res4c = ResId(prev=1024, f1=256, f2=1024)
        self.res4d = ResId(prev=1024, f1=256, f2=1024)
        
        self.conv4e = Conv(in_f=1024, k=3, s=1, out_f=512, p=1)
        self.conv4f = Conv(in_f=512, k=3, s=1, out_f=256, p=1)
        
        
        '''
        wasn't in the paper just for testing FC delay
        '''
        self.fc1 = nn.Linear(921600, 1000) 
        self.fc2 = nn.Linear(1000, 128) 
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.res2a(x)
        x = self.res2b(x)
        x = self.res2c(x)
        
        x = self.res3a(x)
        x = self.res3b(x)
        x = self.res3c(x)
        
        x = self.res4a(x)
        x = self.res4b(x)
        x = self.res4c(x)
        x = self.res4d(x)
        
        x = self.conv4e(x)
        x = self.conv4f(x)
            
        '''
        wasn't in the paper just for testing FC delay
        '''
        x = x.view((1,-1))
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [0]:
model = Model()

In [0]:
model.cuda()

Model(
  (conv1): Conv(
    (conv): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res2a): ResConv(
    (conv1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): Conv(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (res): Conv(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, 

In [0]:
model.eval()

Model(
  (conv1): Conv(
    (conv): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res2a): ResConv(
    (conv1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): Conv(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (res): Conv(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, 

In [0]:
rgbd_img = torch.rand((4,480,480))
rgbd_img = rgbd_img.cuda()

In [0]:
i = 0
with torch.no_grad():
  
    t = time()
    while True:
        i+=1
        model(rgbd_img[None])
        
        if time()-t>1:      
            print("time: {} s".format(time()-t))
            print("number of feedforward: {}".format(i))
            break

time: 1.0398070812225342 s
number of feedforward: 19


In [0]:
!nvidia-smi

Sat May  4 19:49:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    29W /  70W |   4587MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  